In [2]:
# Imports
import pandas as pd
from path import Path
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
# Read csv file
cc_transactions_df = pd.read_csv(Path("Resources/credit_card_transactions.csv"))

cc_transactions_df.head()

,dateMonth,dateWeekday,dateQuarter,operationType,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,1,1,4,736.76,1120.00,383.24,0.00,0.00,0
1,1,1,1,2,256226.20,0.00,0.00,518041.64,1305828.01,0
2,1,1,1,5,310027.49,61594.00,0.00,10089.00,1157398.83,0
3,1,1,1,1,76695.35,9064440.33,9141135.68,880045.33,206147.08,0
4,1,1,1,4,4761.57,0.00,0.00,0.00,0.00,0


In [8]:
# Define target set (y)
y = cc_transactions_df['isFraud']

# Y sample
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: isFraud, dtype: int64

In [10]:
# Define features set (X)
X = cc_transactions_df.drop(columns=["isFraud"])

# X sample
X.head()

,dateMonth,dateWeekday,dateQuarter,operationType,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,1,1,4,736.76,1120.00,383.24,0.00,0.00
1,1,1,1,2,256226.20,0.00,0.00,518041.64,1305828.01
2,1,1,1,5,310027.49,61594.00,0.00,10089.00,1157398.83
3,1,1,1,1,76695.35,9064440.33,9141135.68,880045.33,206147.08
4,1,1,1,4,4761.57,0.00,0.00,0.00,0.00


In [11]:
# Create training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
# Create scaler instance
X_scaler = StandardScaler()

In [15]:
# Fit scaler 
X_scaler.fit(X_train)

# Scale training and testing data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Define Neural Network # 

# numner of inputs
number_input_features = len(X_train.iloc[0])

# hidden layer nodes
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Sequential instance
nn = Sequential()

# add input layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# add layer 2
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# add output layer
nn.add(Dense(1, activation="sigmoid"))


In [25]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 5)                 50        
                                                                 
 dense_10 (Dense)            (None, 3)                 18        
                                                                 
 dense_11 (Dense)            (None, 1)                 4         
                                                                 
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Fit the model 
model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2344/2344 [==============================] - 2s 782us/step - loss: 0.0312 - accuracy: 0.9988
Epoch 2/100
2344/2344 [==============================] - 2s 776us/step - loss: 0.0099 - accuracy: 0.9988
Epoch 3/100
2344/2344 [==============================] - 2s 780us/step - loss: 0.0093 - accuracy: 0.9988
Epoch 4/100
2344/2344 [==============================] - 2s 796us/step - loss: 0.0088 - accuracy: 0.9988
Epoch 5/100
2344/2344 [==============================] - 2s 788us/step - loss: 0.0085 - accuracy: 0.9988
Epoch 6/100
2344/2344 [==============================] - 2s 784us/step - loss: 0.0081 - accuracy: 0.9988
Epoch 7/100
2344/2344 [==============================] - 2s 783us/step - loss: 0.0079 - accuracy: 0.9988
Epoch 8/100
2344/2344 [==============================] - 2s 779us/step - loss: 0.0077 - accuracy: 0.9988
Epoch 9/100
2344/2344 [==============================] - 2s 787us/step - loss: 0.0076 - accuracy: 0.9988
Epoch 10/100
2344/2344 [==============================]

In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 - 1s - loss: 0.0050 - accuracy: 0.9989 - 505ms/epoch - 646us/step
Loss: 0.004991254769265652, Accuracy: 0.9988800287246704


In [31]:
# Set the model's file path
file_path = Path("./employee_attrition.h5")

# Export your model to a HDF5 file
nn.save(file_path)